In [2]:
import textract
import os
import pandas as pd
import re

import nltk
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

#in case punkt package is missing decomment the line below
#nltk.download("punkt")
#nltk.download('stopwords')


datadir = r"C:\Users\Cheikh\Desktop\Projet_memoire\myArmAi\samples\base_cv\cv"

def loadData(path):
    
    if not os.path.isdir(path):
            raise Exception("OpenFolderException","The given path is not a valid folder or folder doesn't exist")
    _dataset = []
    raw_dataset = [textract.process(os.path.join(datadir,f)).decode() for f in os.listdir(datadir)]
    
    print("Dataset size : {}".format(len(raw_dataset)))
    
    for d in raw_dataset:
        _dataset.append(d)
    return _dataset;


def dataSetInfo(data):
    word_count = 0;
    for _d in data:
        word_count += len(_d)
    return word_count;

dataset = loadData(datadir)


Dataset size : 33


In [4]:
import re
import json
import numpy as np

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import Counter 



train,test =  dataset[0:11],dataset[11:]

#2 tokenize : remove all no alphanumeric character and remove stopwords
tokenizer = RegexpTokenizer(r'\w+')

def dataCleaning(raw_data):
    cleaned_data = []
    stop_words = set(stopwords.words("french"))
    for data in raw_data:
        cleaned_data.append([x.lower() for x in tokenizer.tokenize(data) if x not in stop_words])
    return cleaned_data;

train = dataCleaning(train)

occurences  = Counter(train[0])

In [5]:
import os
path = r"C:\Users\Cheikh\Desktop\Projet_memoire\myArmAi\samples\cv\cv_atos\fr\word"

def renameDoc2Docx(path):
    dir = os.listdir(path)
    for d in dir:
        file = os.path.join(path,d)
        if(os.path.isfile(file)):
            name,extension = file.rsplit(".")
            if(extension == "doc"):
                new_name=name + '.docx'
                os.rename(file,new_name)
                

In [11]:
from glob import glob
import re
import os
import win32com.client as win32
from win32com.client import constants

paths = glob(r'C:\Users\Cheikh\Desktop\Projet_memoire\myArmAi\samples\base_cv\cv\**\*.doc', recursive=True)
paths

def save_as_docx(path):
    # Opening MS Word
    word = win32.gencache.EnsureDispatch('Word.Application')
    doc = word.Documents.Open(path)
    doc.Activate ()

    # Rename path with .docx
    new_file_abs = os.path.abspath(path)
    new_file_abs = re.sub(r'\.\w+$', '.docx', new_file_abs)

    # Save and Close
    word.ActiveDocument.SaveAs(
        new_file_abs, FileFormat=constants.wdFormatXMLDocument
    )
    doc.Close(False)

for path in paths:
    save_as_docx(path)

In [43]:
from docx import Document
import pandas as pd

test_file = r"C:\Users\Cheikh\Desktop\Projet_memoire\myArmAi\samples\base_cv\cv\CV ATOS Dionsaba DIEDHIOU.docx"

test_path = r"C:\Users\Cheikh\Desktop\Projet_memoire\myArmAi\samples\base_cv\cv"

dir = [os.path.join(test_path,f) for f in os.listdir(test_path)]
section = ["Formation","Langue"]

def readFile():
    data = []
    for d in dir:
        filename,extension = d.rsplit(".")
        if not re.search(r'\$',filename): ## in windows system not checking this may cause error if one the files have been opened
            if(extension == "docx"):
                document = Document(d)
                table = document.tables[0]
                res = tableToDF(document)
                data.append(res)
    return data

#transform word table to pandas dataframe
def tableToDF(wordDoc):
    tables = []
    for table in wordDoc.tables:
        df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
        for i, row in enumerate(table.rows):
            for j, cell in enumerate(row.cells):
                if cell.text:
                    df[i][j] = cell.text
        tables.append(pd.DataFrame(df))
    return tables;

def dropSpace(s,replace=""):
    return re.sub(r'\n',replace,s)

df = readFile()


In [46]:
test = df[0][4].iloc[1].tolist()
skills_ar = []  
tmps = []
for i in range(1,len(df[0][4])):
    skills_ar.append(df[0][4].iloc[i].tolist())
for ar in skills_ar:
    for el in ar:
        tmps.append(el)
        

    

def getExp(test):
    ## remove empty field
    test = [x for x in test if len(x)!=0]
   ## print(test)
    ##remove duplicate entry
    test = list(dict.fromkeys(test))
    tmp = test
    for i in range(len(test)):
        #print(test[i])
        try:
            v = float(test[i])
            tmp[i] = v
        except:
            pass
        
    test = tmp
    skills ={}
    categories=[]
    for i in range(len(test)):
        if(isinstance(test[i],str)):
            if i+1 <len(test):   
                if(isinstance(test[i+1],str)):
                    categories.append(test[i])
                    val = test[i+1]
                    print(categories)
                if(isinstance(test[i+1],float)):
                    s = test[i]
                    val = test[i+1]
                    ##print(s,val)
                    skills[s] = val
    return skills

getExp(tmps)

['Langage/ Framework / BDD']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython', 'Intégration Html CSS']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython', 'Intégration Html CSS', 'Java / Java JEE']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython', 'Intégration Html CSS', 'Java / Java JEE', 'PHP / Bootstrap']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython', 'Intégration Html CSS', 'Java / Java JEE', 'PHP / Bootstrap', 'Angular']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython', 'Intégration Html CSS', 'Java / Java JEE', 'PHP / Bootstrap', 'Angular', 'Symfony']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython', 'Intégration Html CSS', 'Java / Java JEE', 'PHP / Bootstrap', 'Angular', 'Symfony', 'Django']
['Langage/ Framework / BDD', 'JavaScript /JQuery\nPython', 'Intégration Html CSS', 'Java / Java JEE', 'PHP / Bootstrap', 'Angular', 'Symfony', 'Django', 'Systèmes d’exploitation'

{'SQL / SQL Server\nPostgresql': 2.0,
 'C#': 2.5,
 'HTML / CSS': 2.5,
 'C/C++                           ': 1.5,
 'Ionic                         ': 1.0}

In [9]:
from docx import Document
import pandas as pd
document = Document(test_file)

tables = []
table = document.tables[4]

for row in table.rows:
    iter_unique_cells(row)

In [32]:
# import re
import string
printable = set(string.printable)


##get cv language
def getLang(df):
    patterns = ["language","langue"]
    dfSize = len(df)
    lang = ""
    for i in range(dfSize):
        _df = df[i]
        for index,row in _df.iterrows():
            if(re.search("language",row[0].lower()) or re.search("langue",row[0].lower())):
                if(len(row) == 3):
                    lang = row[2]
                elif len(row) == 2:
                    lang = row[1]
                print("###############")
    lang = ''.join(filter(lambda x: x in printable, lang))
    lang = dropSpace(lang,replace=' ')
    return lang

def getNameAndProfil(df):
    data = df[0]
    name = ''
    profil= ''
    for i in data:
        size = len(data[i])
        if size>0 and size<5:
            try:
                if(int(data[i].str.index("("))): # contain parenthesis xxx (X olds) XXX
                    
                    profil_ind = int(data[i].str.index(")"))
                    name_ind = int(data[i].str.index("("))
                    val = data[i][0]
                    
                    name = val[0:name_ind-1]
                    profil = val[profil_ind+1:]

                       ## val = re.sub("[^\w]"," ",val)
                    name = re.sub("[\\r\\n\|\\t|[0-9]]*", "", name)
                    ##profil = re.sub("[\\r\\n\|\\t|[0-9]]*", "", profil)
                    profil = [x for x in profil.split("\n") if len(x)>0]
                    #print(len(profil))
                   
                    if len(profil) == 2 :
                       
                        if re.search(r'[^0-9]\s',profil[1]): ## search a string that doesn't contain any number followed by space
                            profil = profil[0]
                        else:
                            profil = profil[0]
                           # print(profil)
                    else:
                        profil = profil[0]
                       # print(profil)

            except:
                    value = data[i].tolist()[0].split("\n\n")
                    if len(value) ==2:
                        name,profil = value
                        name = dropSpace(name)
                        profil = re.sub("\t","",profil)
                        #print(profil)
                    elif len(value[0]) != 0: ## some cv may contain nom prenom and profil
                        val = value[0].split(":")
                        if len(val)>2:
                            name,profil = val[1].split("\n")[0],val[2]
                            #print(name,profil)
                    pass
            
    name = name.strip()
    profil = profil.strip()
    return name,profil
        

result = getNameAndProfil(df[0])
print(result)

##for d in df:
##    res = getNameAndProfil(d)
  #  print(res)


('Abdoulaye BARRO', 'Ingénieur Etude et développement')


In [16]:
def getFormation(data):
    pattern = ["formation","training"]
    dfSize = len(data)
    formation = ""
    
    for i in range(dfSize):
        _data = data[i]
        
        for index,row in _data.iterrows():
            
            if(re.search(pattern[0],row[0].lower()) or re.search(pattern[1],row[0].lower())):
                if(len(row) == 3):
                    formation = row[2]
                    
    formation = dropSpace(formation," ")
    return formation
        
res = getFormation(df[7])
res

'2009, Maitrise en informatique, Université Gaston Berger de Saint Louis  2007, Licence en informatique Université Gaston Berger de Saint Louis (UGB/SL)              2006, Diplôme d’Etude Universitaire Général (DEUG II) en Mathématiques Physique et Informatique (MPI) (UGB/SL)  2005, Diplôme d’Etude Universitaire Général (DEUG I) en Mathématiques Physique et Informatique (MPI) (UGB/SL)   2002, BACCALAUREAT Série S2 au Lycée Djignabo de Ziguinchor    '

In [17]:
def getExperience(data):
    pattern = ["experience","expérience"]
    dfSize = len(data)
    experience = ""
    for i in range(dfSize):
        _data = data[i]
        for index,row in _data.iterrows():
            if len(row) <= 3 :
                for el in row:
                    if re.search(pattern[0],el.lower()) or re.search(pattern[1],el.lower()):
                        el = re.sub(r'\((.*?)\)','',el)
                        res = re.search(r'[0-9]{1,2}\s',el)
                        if res != None:
                            begin,end = res.span()
                            experience = el[begin:end]
                            #print(experience)
    return experience

for d in df:
    name = getNameAndProfil(d)
    r = getExperience(d)
    print(name,r)
    

('Abdoulaye BARRO', 'Ingénieur Etude et développement') 
('Alassane SANGOTT', 'Ingénieur Conception et Réalisation') 9 
('Amadou NDIAYE', 'Research and Development Engineer') 
('Balla DIENG', 'Ingénieur Etudes & Développement') 10 
('Lo Bathie Massamba', 'Ingénieur Etude et Développement') 3 
('Cedric Mukanya Malumba', 'Consultant Technique Oracle') 10 
('Demba Kalidou SOW', 'Professional Scrum Master') 6 
('Dionsaba DIEDHIOU', 'Ingénieur étude et développement') 8 
('Fassou Mathias NIAMY', 'Software Engineer') 
('', '') 
('Hawa Evelyne BA', 'DEVELOPMENT ENGINEER') 
('Hawa Evelyne BA', 'INGENIEUR INFORMATIQUE DECISIONNEL') 
('Ibrahima GUEYE', 'Ingénieur Etude et Développement') 3 
('Lo Bathie Massamba', 'Ingénieur Etude et Développement') 3 
('Akoko Martine TOWANOU', 'Ingénieur Etude et Développement') 
('Momath NDIAYE', 'Ingénieur Étude et Développement') 
('Mouhamadou Al Moustapha DIOP', 'Développeur Fullstack, BI & Big Data') 
('Mouhamadoul Mansour SAMB', 'Chef de Projet') 10 
('', 

In [47]:
from collections import Counter

def getSkills(data):
    pattern = ["compétences","skills"]
    dfSize = len(data)
    competences = {}
    _max = max(Counter([len(x) for x in data]).keys())
    print(_max)

    for i in range(dfSize):
        _data = data[i]
        for index,row in _data.iterrows():
            ## find principales and fonctionnale competences
            for el in row:
                if re.search(pattern[0],el.lower()) or re.search(pattern[1],el.lower()):
                    value = [x for x in row.tolist() if len(x)!=0]
                    out =""
                    for s in value:
                        if(not re.search(pattern[0],s.lower())):
                            out+=s+u" "
                    competences[el] = out

    competences = {x:dropSpace(y.replace("\xa0",""),replace=' ') for x, y  in competences.items()}
    ##print(competences)
    ## find technical competences
    ## we assume that array containing technical skills have the same length
    ##print(row)
    skills = [x for x in data if len(x)==_max][0]
    return skills

            
            
skills = getSkills(df[3])
skills

18


,0,1,2,3,4
0,Chaka Computer,Chaka Computer,,,
1,Mars.2009 à Sep 2009,Mars.2009 à Sep 2009,,,
2,Fonction\n,Fonction\n,,,Ingénieur Etudes et Développement
3,Projet\n,Projet\n,,,Analyse et développement autour des applications bancaires et postales\n
4,Mission(s) et réalisations\n,Mission(s) et réalisations\n,,,Internationalisation d’application \nFinancia : logiciel de micro finance\nPOSTALIA : Logiciel de gestion des produits postaux\nDéveloppement d’une application bancaire : DACIBANK\nSuivi et tests de logiciel existants.\n
5,Environnement\n,Environnement\n,,,"Eclipse, Java/JEE, Struts, JSP, Oracle, SVN"
6,Convergence Informatique,,,,
7,Juil. 2008 à Jan 2009,,,,
8,Fonction\n,,,Ingénieur Etudes et Développement(Stagiaire),Ingénieur Etudes et Développement(Stagiaire)
9,Projet\n,,,Développement d’un portail de données statistiques pour l’ANSD (Agence nationale de la statistique et de la démographie)\t,Développement d’un portail de données statistiques pour l’ANSD (Agence nationale de la statistique et de la démographie)\t
